In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
# Any results you write to the current directory are saved as output.

In [ ]:
from io import StringIO

Nvals = 4
Frac = 0.05

# Tells notebook to display all rows or columns instead of truncating
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

# read_random()
#
# Quickly read a dataframe from a file.  Assumes that there is a header.
# Reading into a buffer, then read_csv from the buffer, is much faster than reading directly from the file
# https://stackoverflow.com/questions/38233719/reading-random-rows-of-a-large-csv-file-python-pandas
#
# writeheader = should ordinarily be set to True.  A case where you might not want to
#      write the header is if you have already recorded it, and you are starting from the middle of the file.
# minrow = first row to read.  (0 = start from beginning.)
# maxrow = max number of rows to read from csv file (rows are from 0 to maxrow - 1)
#          it's okay if the file contains fewer than maxrow rows.
# prob = probability of including each row
# num_reports = number of times to report progress
# kwargs = to be passed to read_csv() function that parses the csv data.
#
def read_random(filename, writeheader = True, minrow = 0, maxrow = 100, prob = 0.12345, num_reports = 10, **kwargs):

    rowcount = -1
    rowcount_div = 0
    start = True
    s_buf = StringIO()
    
    with open(filename) as file:
        for line in file:
            if rowcount == -1: # get header
                rowcount += 1
                if writeheader:
                    s_buf.write(line)
            elif rowcount < minrow: # skip all rows before minrow
                rowcount += 1
                continue
            elif rowcount >= maxrow: # stop at maxrow
                break
            else: # in between minrow and maxrow
                rowcount += 1
                if np.random.rand() < prob:
                    s_buf.write(line)
                if rowcount_div >= (maxrow - minrow) // num_reports - 1:
                    print("At row: {}".format(rowcount))
                    rowcount_div = 0
                else:
                    rowcount_div += 1
    print("Done!  Rowcount is: {}".format(rowcount))
    s_buf.seek(0)
    return pd.read_csv(s_buf, **kwargs)

In [ ]:
from collections import OrderedDict
def make_dummies_from_df_train(df_train, Nvals = 5):

    cols = list()
    dm = None
    cat_columns = OrderedDict()
    expand_list = ('EngineVersion', 'AppVersion', 'AvSigVersion', 'OsVer', 'OsBuildLab', 'Census_OSVersion')
    # TEST expand_list = ('A')
    
    for col in df_train.columns:
        if col in expand_list:
            df_train = pd.concat((df_train, col_split(df_train, col)), axis=1)
        
    # This loop goes through all columns in df_train of types "object"
    # In ech column, it sets cat_columns[col] to a list of the
    # Nvals most common values (or less, if less values exist.)
    # It then gets the dummies for each column for those NVals values and concatenates them.
    # The return value is the concatenated dummy values.
    for col in df_train.columns:
        if col in expand_list:
            continue
        if df_train.dtypes[col] == 'object':
            # We actually process each column twice:
            # once to get the value counts to see which are the three most common values
            # and once to make a get_dummies version of it.  cat_columns has to store
            # the ordering produced by the latter.
            s = df_train[col].value_counts()
            if s.size > 1000:
                print("Column {} is long; skipping it.".format(col))
                continue
            if s.size > Nvals:
                s_temp = df_train[col].replace(list(s.iloc[Nvals:].index), np.nan, inplace=False)
            else:
                s_temp = df_train[col]
            dmcol = pd.get_dummies(s_temp, prefix = col, prefix_sep="_")
            vals = [x[len(col) + 1:] for x in list(dmcol.columns)]
            cat_columns[col] = vals
            if dm is None:
                dm = dmcol
            else:
                dm = pd.concat([dm, dmcol], axis=1)
            del dmcol
    return dm, cat_columns

In [ ]:
# Pipeline: scale and impute data
from sklearn.preprocessing import StandardScaler

# Set stdscale to true if you want to standardize the data and fill nans with zero.
# Set stdscale to false if you just want to fill nans with -1 and leave the scaling alone.
def ScaleImpute(X, dummy_frame, stdscale = True):
    # numer_columns = list(X.columns)
    # X_num is a numerical only version of X, where categorical columns have been converted
    # to dummies.  It may be scaled or unscaled.
    if stdscale:
        scaler = StandardScaler()
        X_num = np.nan_to_num(scaler.fit_transform(pd.concat([X.astype(float), dummy_frame.astype(float)], axis = 1)))
    else:
        X_num = pd.concat([X.astype(float).fillna(-1), dummy_frame.astype(float)], axis = 1)
    return X_num

from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

def test_roc_auc(X_num, y, clf):
    X_train, X_test, y_train, y_test = train_test_split(X_num, y, test_size=0.33, random_state=42)

    #sgd_clf = SGDClassifier(loss = 'log', penalty='l1')
    #sgd_clf.fit(X_num, y)

    print("About to fit.")
    clf.fit(X_train, y_train)
    print("About to predict_proba()")
    y_predict = clf.predict_proba(X_test)
    print("About to compute roc_auc")
    print("roc_auc:{}".format(roc_auc_score(y_test, y_predict[:,1])))
    return(clf)

In [ ]:
def getnth(x, n):
    try:
        return x.split('.')[n]
    except Exception:
        print("Warning: {} cannot be split; part #{} will be NULL.".format(x, n))
        return "NULL"

def col_split(df, col):
    lst = list()
    names = list()
    numcols = len(df[col].iloc[0].split('.'))
    for n in range(0, numcols):
        lst.append(df[col].apply(lambda x: getnth(x, n)).astype('object').rename(col + str(n)))
        names.append(col + str(n))
    return pd.concat(lst, axis = 1)

In [ ]:
os.system('echo read_random')
df_train = read_random("../input/train.csv", minrow = 0, maxrow = 20000000, prob = Frac)

In [ ]:
# TEST df_train = pd.DataFrame({'A': ['1.2.3', '4.5.6', '7.8.9'], 'B': [4, 5, 6], 'HasDetections': [1, 1, 0]})

In [ ]:
os.system('echo make_dummies_from_df_train')
dummy_frame, cat_columns = make_dummies_from_df_train(df_train, Nvals = Nvals)
#for k, v in cat_columns.items():
#    print(k, v)

In [ ]:
#dummy_frame

In [ ]:
cat_columns

In [ ]:
X = df_train.drop('HasDetections', axis = 1)
numer_columns = X.select_dtypes(exclude=['object']).columns
y = df_train.HasDetections

In [ ]:
numer_columns

In [ ]:
X_num = ScaleImpute(X.select_dtypes(exclude=['object']), dummy_frame, stdscale = True)

In [ ]:
X_num

In [ ]:
import scipy.stats

lst = list()
tot = 0

# This code could be used to test whether columns M and N in the dummies
# correspond to the same feature, i.e.
# get_feature_number(div_columns, N) == get_feature_number(div_columns, M)
# I don't think it's necessary to check this, however.  These products aren't
# "cheating" in any sense, and there are relatively few of them.
#div_columns = [1 for x in range(len(numer_columns))] + [len(x) for x in cat_columns.values()]
#def get_feature_number(div_columns, N):
#    dc_cpy = div_columns.copy()
#    total = 0
#    feature_number = 0
#    while total < N:
#        total += dc_cpy.pop(0)
#        feature_number += 1
#    return feature_number

os.system('echo compute correlations')
# start: 2:38 PM
# Go through all pairs (N, M) with N >= M
# Multiply feature N by feature M elementwise to get the interaction term columns
# Find the interaction term column's correlation with HasDetections
for N in range(0, X_num.shape[1]):
    for M in range(0, N + 1):
        c = scipy.stats.pearsonr(X_num[:,N] * X_num[:,M], df_train.HasDetections)[0]
        if pd.notna(c):
            #tot += 1
            # if tot > 1000:
            #    break
            lst.append([N, M, c])
    #if tot > 1000:
    #    break

In [ ]:
os.system('echo concatenate correlated columns')
N_pairs = 100
# TEST N_pairs = 2
pct = np.percentile(np.array([np.abs(x[2]) for x in lst]), 100 * (1 - N_pairs / len(lst)))
# TEST: should be >= pct
interaction_triples = [x for x in lst if np.abs(x[2]) > pct]
#X_num_cpy = X_num.copy()
for x in interaction_triples:
    X_num = np.concatenate((X_num, np.reshape(X_num[:, x[0]] * X_num[:, x[1]], (-1, 1))), axis = 1)

In [ ]:
X_num

In [ ]:
interaction_triples

In [ ]:
#from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

#parameters = {'C':2.0**-(np.arange(1))}
#logreg = LogisticRegression(penalty='l1', solver='liblinear')
#clf = GridSearchCV(logreg, parameters, cv=2)
#clf.fit(X_num, y)
os.system('echo LogisticRegression')
# TEMP clf = LogisticRegression(penalty='l1', solver='liblinear', C=2**(-7))
clf = RandomForestClassifier(n_estimators = 300)

clf = test_roc_auc(X_num, y, clf)

In [ ]:
from joblib import dump, load
dump(clf, 'clf.joblib')
dump(numer_columns, 'numer_columns.joblib')
dump(cat_columns, 'cat_columns.joblib')
dump(interaction_triples, 'interaction_triples.joblib')
# clf = load('filename.joblib') 

In [ ]:
#print(interaction_triples)

In [ ]:
# You can't use make_dummies_from_df to process X_test, because make_dummies_from_df
# expects to take a df in a format read directly from the file.  X_test has already been
# processed into numerical and categorical / dummy columns.

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
#rf_clf = RandomForestClassifier()
#rf_clf.fit(X_train, y_train)
#y_rf_predict = rf_clf.predict_proba(X_test)

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
#rf_clf = RandomForestClassifier()
#rf_clf.fit(X_num, y)

In [ ]:
#print(rf_clf.classes_) # If [0 1], use column 1 for HasDetections probability
# otherwise, use column 0

In [ ]:
#decide_test_file("../input/test.csv", "out_sm.csv", sgd_clf, numer_columns, cat_columns, num_reports = 5, numrows=200000, stdscale=stdscale)

# 

In [ ]:
#7853254